In [7]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score

df = pd.read_csv('Energy.csv')
df1 = pd.DataFrame(df)

#Spliting into input and target variables
X = df1.drop(columns=['Heating_Load', 'Cooling_Load'])
y_Heating = df1['Heating_Load']
y_Cooling = df1['Cooling_Load']

# Split the dataset for heating_load prediction
xtrain_heat, xtest_heat, ytrain_heat, ytest_heat = train_test_split(X, y_Heating, test_size=0.25, random_state=0)

# Split the dataset for cooling_load prediction
xtrain_cooling, xtest_cooling, ytrain_cooling, ytest_cooling = train_test_split(X, y_Cooling, test_size=0.25, random_state=0)

# Define and train the model for drug prediction
model_heat = lgb.LGBMRegressor()
model_heat.fit(xtrain_heat, ytrain_heat)

# Define and train the model for dosage prediction
model_cool = lgb.LGBMRegressor()
model_cool.fit(xtrain_cooling, ytrain_cooling)



[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000030 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 57
[LightGBM] [Info] Number of data points in the train set: 576, number of used features: 8
[LightGBM] [Info] Start training from score 22.147899
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -in

LGBMRegressor()

In [10]:
# Predicting on the test data
y_pred_heat = model_heat.predict(xtest_heat)
y_pred_cool = model_cool.predict(xtest_cooling)

# Calculating R^2 score for the heat model
score_heat = r2_score(ytest_heat, y_pred_heat)

# Calculating R^2 score for the cooling model
score_cool = r2_score(ytest_cooling, y_pred_cool)

print("R^2 Score for test Heat Model:", score_heat)
print("R^2 Score for test Cooling Model:", score_cool)

y_pred_heat1 = model_heat.predict(xtrain_heat)
y_pred_cool1 = model_cool.predict(xtrain_cooling)

# Calculating R^2 score for the heat model
score_heat = r2_score(ytrain_heat, y_pred_heat1)

# Calculating R^2 score for the cooling model
score_cool = r2_score(ytrain_cooling, y_pred_cool1)

print("\nR^2 Score for train Heat Model:", score_heat)
print("R^2 Score for train Cooling Model:", score_cool)

R^2 Score for test Heat Model: 0.9969313729837492
R^2 Score for test Cooling Model: 0.9805535265844246

R^2 Score for train Heat Model: 0.9983054561779183
R^2 Score for train Cooling Model: 0.9909185645465438


In [14]:
# Evaluate the heating load model
print("Heating Load Model Evaluation:")
print(f"R-squared: {r2_score(ytest_heat, y_pred_heat):.2f}")
print(f"Mean Absolute Error: {mean_absolute_error(ytest_heat, y_pred_heat):.2f}")
print(f"Mean Squared Error: {mean_squared_error(ytest_heat, y_pred_heat):.2f}")


# Evaluate the cooling load model
print("\nCooling Load Model Evaluation:")
print(f"R-squared: {r2_score(ytest_cooling, y_pred_cool):.2f}")
print(f"Mean Absolute Error: {mean_absolute_error(ytest_cooling, y_pred_cool):.2f}")
print(f"Mean Squared Error: {mean_squared_error(ytest_cooling, y_pred_cool):.2f}")

Heating Load Model Evaluation:
R-squared: 1.00
Mean Absolute Error: 0.38
Mean Squared Error: 0.33

Cooling Load Model Evaluation:
R-squared: 0.98
Mean Absolute Error: 0.92
Mean Squared Error: 1.80


In [16]:
# Define thresholds to categorize the heating and cooling loads
def categorize_load(value):
    if value < 15:
        return 'Low'
    elif value < 30:
        return 'Medium'
    else:
        return 'High'

# Apply the thresholds to categorize the actual and predicted values
ytest_heat_cat = ytest_heat.apply(categorize_load)
y_pred_heat_cat = pd.Series(y_pred_heat).apply(categorize_load)

ytest_cooling_cat = ytest_cooling.apply(categorize_load)
y_pred_cool_cat = pd.Series(y_pred_cool).apply(categorize_load)

# Evaluate the heating load classification
print("\nHeating Load Classification Evaluation:")
print("Confusion Matrix:")
print(confusion_matrix(ytest_heat_cat, y_pred_heat_cat))
print("\nClassification Report:")
print(classification_report(ytest_heat_cat, y_pred_heat_cat))



Heating Load Classification Evaluation:
Confusion Matrix:
[[53  0  0]
 [ 0 76  0]
 [ 0  0 63]]

Classification Report:
              precision    recall  f1-score   support

        High       1.00      1.00      1.00        53
         Low       1.00      1.00      1.00        76
      Medium       1.00      1.00      1.00        63

    accuracy                           1.00       192
   macro avg       1.00      1.00      1.00       192
weighted avg       1.00      1.00      1.00       192



In [17]:
# Evaluate the cooling load classification
print("\nCooling Load Classification Evaluation:")
print("Confusion Matrix:")
print(confusion_matrix(ytest_cooling_cat, y_pred_cool_cat))
print("\nClassification Report:")
print(classification_report(ytest_cooling_cat, y_pred_cool_cat))


Cooling Load Classification Evaluation:
Confusion Matrix:
[[65  0  6]
 [ 0 33  5]
 [ 6  5 72]]

Classification Report:
              precision    recall  f1-score   support

        High       0.92      0.92      0.92        71
         Low       0.87      0.87      0.87        38
      Medium       0.87      0.87      0.87        83

    accuracy                           0.89       192
   macro avg       0.88      0.88      0.88       192
weighted avg       0.89      0.89      0.89       192



In [18]:
# Prompt the user for input values
input_feature_names = X.columns.tolist()
user_input = []
print("\nPlease enter the following details:")
for feature in input_feature_names:
    value = float(input(f"{feature}: "))
    user_input.append(value)

# Convert user input to appropriate types
features = np.array([user_input])

# Make predictions based on user input
prediction = model_heat.predict(features)
prediction2 = model_cool.predict(features)

print("\nThe required heating load is: {:.2f} KW/H".format(prediction[0]))
print("The required cooling load is: {:.2f} KW/H".format(prediction2[0]))


Please enter the following details:
Relative_Compactness: 0.98
Surface_Area: 514.5
Wall_Area: 110.25
Roof_Area: 7
Overall_Height: 2
Orientation: 0
Glazing_Area: 0
Glazing_Area_Distribution: 0

The required heating load is: 17.44 KW/H
The required cooling load is: 23.53 KW/H
